In [102]:
# !pip install --upgrade pip
# !pip install langchain_google_community langchain_huggingface sentence-transformers langchain chromadb pypdf faiss-cpu  langchain_community scikit-learn matplotlib seaborn numpy mistralai langchain-mistralai langchain_classic

langchain
langchain_huggingface
langchain_community
langchain_classic
langchain-mistralai
sentence-transformers
mistralai
pypdf
chromadb
faiss-cpu
scikit-learn
seaborn
numpy
pandas



In [2]:
import os
from getpass import getpass
from google.colab import drive
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS,Chroma
from langchain_classic.schema import Document
from langchain_mistralai import ChatMistralAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains import create_retrieval_chain
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFDirectoryLoader, GithubFileLoader
from langchain_google_community import GoogleDriveLoader,GoogleSearchAPIWrapper


In [18]:
import json
json_keys={}
with open("./keys.json","r") as f:
  json_keys=json.load(f)

In [17]:

os.environ["GOOGLE_API_KEY"] = json_keys["googlesearchapi"]
os.environ["GOOGLE_CSE_ID"] = json_keys["cseapi"]
os.environ["MISTRAL_API_KEY"]=json_keys["mistralapi"]

# Data Loader

In [37]:
loader = GoogleSearchAPIWrapper()
search_list=["what is debloater ?", "what is machine learning ?", "what is the top job notifications on IT roles from government sector ?", "how to remove google apps from android os using adb tools ?", "what is shizuku ?", "how to optimize the wondows 11 to speed up windows os ?"]
search_result = {}

for sq in search_list:
  search_result[sq]=loader.run(sq)

# print(search_result)

# for i in range(0,len(search_result),300):
#   data.append(Document(page_content=search_result[i:i+300]))

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100, separators=["..","...",".","\n\n"])
chunks=splitter.create_documents(search_result.values())

print(len(chunks))


25


In [38]:
# embeddings
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# vectordb use chroma or FAISS
vectordb = FAISS.from_documents(chunks, embedding)


In [40]:
# vectordb.save_local("faiss_index")
# vectordb.search(search_type="similarity",query="what is langchain ?")

In [51]:
## retrive only top k data from vector db
retriver = vectordb.as_retriever(search_kwargs={'k':3})

## llm we use for our RAG
llm = ChatMistralAI(model_name="mistral-tiny")


# Q2Q Transformations


1.   have a query
2.   generate a more pro query with existing query
     with the help of ai agent. [query to query transformations]



In [56]:
## Template that we use for requery by llm
prompt = ChatPromptTemplate.from_template('''
You are a query optimization assistant.

Rewrite the user query so that it retrieves the most relevant document chunks.
Make it specific and explicit.

User Query:
{query}

Optimized Query:
''')

query = "what debloating actual is on android os, with indetailed answer ?"

optimized_q2q = llm.invoke(prompt.format(query=query)).content
print(optimized_q2q)

"Explain in detail the concept and implementation of debloating in the Android operating system, providing a comprehensive answer."


In [59]:
answer_prompt = ChatPromptTemplate.from_template(
    '''
    Answer the following question based only on the provided context:

    <context>
    {context}
    </context>

    Question: {input}
    '''
)

llm_chain = create_stuff_documents_chain(llm,answer_prompt)
res = llm_chain.invoke({"input":optimized_q2q,"context":retriver.invoke(optimized_q2q)})
print(res)

In the context provided, "debloating" in the Android operating system is primarily associated with the Universal Android Debloater (UAD), a graphical user interface (GUI) for Android Debug Bridge (ADB). UAD allows users to remove pre-installed or unwanted apps, system services, and bloatware from their Android devices, even if they are non-rooted.

The implementation of debloating in Android devices using UAD involves the following steps:

1. Connecting the Android device to a computer: The user connects their Android device to the computer using a USB cable.

2. Enabling Developer Options and USB Debugging: The user enables Developer Options on their Android device and allows USB Debugging. This allows the computer to communicate with the device.

3. Installing UAD and enabling ADB: The user installs UAD on their computer and sets up ADB on their device. This can be done by running the command `adb devices` in the terminal or command prompt on the computer to verify the connection.

4

# Multi Hop Query Transformation -

1.   divides sub query
2.   each sub query has its own answer part
3.   combined answer and summarize it



# 1. existing query

In [60]:
print(query)

what debloating actual is on android os, with indetailed answer ?


# 2. dividing query into sub queries using llm

In [63]:
multi_hop_prompt = ChatPromptTemplate.from_template("""
Break the following question into multiple smaller sub-questions.
Each sub-question should retrieve different information.

Return them as a just questions format.

Question:
{query}
""")

sub_queries = llm.invoke(multi_hop_prompt.format(query=query)).content
print(sub_queries)

1. What is debloating in the context of Android OS?

2. What are the specific components that are typically targeted when debloating Android OS?

3. How does the process of debloating Android OS work? (e.g., disabling, uninstalling, or removing system apps)

4. What are the benefits of debloating Android OS? (e.g., improved performance, increased battery life, and enhanced user privacy)

5. What potential drawbacks or risks are associated with debloating Android OS? (e.g., system instability, loss of functionality, or voiding the warranty)

6. What tools or methods are commonly used for debloating Android OS? (e.g., third-party apps, custom recovery modes, or rooting)

7. Are there any recommended precautions to take before attempting to debloat Android OS? (e.g., backing up data, understanding the device's specific system architecture)

8. What are the potential long-term impacts of debloating Android OS on the user experience and the device's overall performance? (e.g., updates, comp

# 2.1 make it as proper list items without numbers and other symbols

In [73]:
sub_query = [q.strip("'.0123456789 ") for q in sub_queries.split("\n") if q!= '' or q.strip()]
print(sub_query)

['What is debloating in the context of Android OS?', 'What are the specific components that are typically targeted when debloating Android OS?', 'How does the process of debloating Android OS work? (e.g., disabling, uninstalling, or removing system apps)', 'What are the benefits of debloating Android OS? (e.g., improved performance, increased battery life, and enhanced user privacy)', 'What potential drawbacks or risks are associated with debloating Android OS? (e.g., system instability, loss of functionality, or voiding the warranty)', 'What tools or methods are commonly used for debloating Android OS? (e.g., third-party apps, custom recovery modes, or rooting)', "Are there any recommended precautions to take before attempting to debloat Android OS? (e.g., backing up data, understanding the device's specific system architecture)", "What are the potential long-term impacts of debloating Android OS on the user experience and the device's overall performance? (e.g., updates, compatibilit

In [71]:
len(sub_query)

8

# 3. Retriving k contexts of each query from sub queries

In [76]:
sub_query_chunks = []
for q in sub_query:
  sub_query_chunks.extend(retriver.invoke(q))
print(len(sub_query_chunks))
print(sub_query_chunks)


24
[Document(id='55168466-3778-4e7e-89cd-dd1bd61d23a0', metadata={}, page_content="... Debloat AI helps you achieve a leaner, more defined face with AI-powered analysis and expert-guided debloating exercises.* Using cutting-edge facial\xa0... Jul 14, 2024 ... What is Universal Android Debloater? It's essentially a GUI interface for Android ADB, allowing you to do things you wouldn't otherwise be able\xa0... Jul 26, 2025 ... Winaero is great for shutting off windows stuff that you don't want and need running. It will not remove stuff though. The best way to debloat\xa0... Sep 24, 2022"), Document(id='64a00cf1-e055-4580-a6aa-90f3dfd9cae3', metadata={}, page_content="Aug 7, 2024 ... I'm just looking to optimize my pc by removing all the extra unnecessary junk but still retaining the tools I want/need. Aug 11, 2021 ... How to Debloat: 8 Simple Steps and What to Know · 1. Drink plenty of water · 2. Consider your fiber intake · 3. Eat less sodium · 4. Be mindful\xa0... Sep 14, 2023 ... The m

# 4. extract unique context which getted from vectordb so that duplicate data will not goes into model and model tokens are wasted unneccessarily

In [83]:
unique_docs = {doc.page_content:doc for doc in sub_query_chunks}

final_docs = list(unique_docs.values())
print(len(final_docs))

4


In [85]:
for doc in final_docs:
  print(doc.page_content)

... Debloat AI helps you achieve a leaner, more defined face with AI-powered analysis and expert-guided debloating exercises.* Using cutting-edge facial ... Jul 14, 2024 ... What is Universal Android Debloater? It's essentially a GUI interface for Android ADB, allowing you to do things you wouldn't otherwise be able ... Jul 26, 2025 ... Winaero is great for shutting off windows stuff that you don't want and need running. It will not remove stuff though. The best way to debloat ... Sep 24, 2022
Aug 7, 2024 ... I'm just looking to optimize my pc by removing all the extra unnecessary junk but still retaining the tools I want/need. Aug 11, 2021 ... How to Debloat: 8 Simple Steps and What to Know · 1. Drink plenty of water · 2. Consider your fiber intake · 3. Eat less sodium · 4. Be mindful ... Sep 14, 2023 ... The main reason people do this is because they say that you can get the best performance and for privacy reasons. Is it really necessary though? Does it really
... Aug 29, 2023 ... C

# 5. testing llm using context as multi hop and query as Optimized one with q2q

In [87]:
res = llm_chain.invoke({"input":optimized_q2q,"context":final_docs})
print(res)

The provided context does not offer a comprehensive explanation of the concept and implementation of debloating in the Android operating system, but it does provide some insights into tools and methods used for debloating.

Debloating in the context of Android refers to the process of removing unwanted or unnecessary applications, system files, and bloatware (pre-installed applications that are not needed or used by the user) to improve the device's performance, security, and battery life.

The context mentions Universal Android Debloater, a GUI interface for Android ADB (Android Debug Bridge) that allows users to perform debloating tasks on non-rooted Android devices. Universal Android Debloater is written in Rust and uses ADB to interact with the Android system.

To debloat an Android device using Universal Android Debloater, users can follow these steps:

1. Install Android SDK Platform-Tools (ADB) on your computer if you haven't already.
2. Download and install Universal Android De